## Minesweeper

### Generate the numbers for safe squares in a Minesweeper grid

Difficulty: *medium* to *hard*

If you've ever used an older version of Windows, there's a good chance you've played with Minesweeper:
- https://en.wikipedia.org/wiki/Minesweeper_(video_game)


If you're not familiar with the game, imagine a grid of squares: some of these squares conceal a mine. If you click on a mine, you lose instantly. If you click on a safe square, you reveal a number telling you how many mines are found in the squares that are immediately adjacent. The aim of the game is to uncover all squares in the grid that do not contain a mine.

In this section, we'll make a DataFrame that contains the necessary data for a game of Minesweeper: coordinates of the squares, whether the square contains a mine and the number of mines found on adjacent squares.

**51**. Let's suppose we're playing Minesweeper on a 5 by 4 grid, i.e.
```
X = 5
Y = 4
```
To begin, generate a DataFrame `df` with two columns, `'x'` and `'y'` containing every coordinate for this grid. That is, the DataFrame should start:
```
   x  y
0  0  0
1  0  1
2  0  2
```

In [1]:
import pandas as pd
import numpy as np

In [2]:
y_size = 4
x_size = 5

df = pd.DataFrame([(x, y) for x in range(x_size) for y in range(y_size)], columns=['x', 'y'])

**52**. For this DataFrame `df`, create a new column of zeros (safe) and ones (mine). The probability of a mine occuring at each location should be 0.4.

In [3]:
_rng = np.random.default_rng(0x52)
df["has_mine"] = _rng.choice([0, 1], p = (0.6, 0.4), size = len(df))
df

,x,y,has_mine
0,0,0,1
1,0,1,1
2,0,2,0
3,0,3,1
4,1,0,0
5,1,1,1
6,1,2,0
7,1,3,0
8,2,0,1
9,2,1,0


**53**. Now create a new column for this DataFrame called `'adjacent'`. This column should contain the number of mines found on adjacent squares in the grid. 

(E.g. for the first row, which is the entry for the coordinate `(0, 0)`, count how many mines are found on the coordinates `(0, 1)`, `(1, 0)` and `(1, 1)`.)

In [4]:
x_curr = 0
y_curr = 0

# Very cool alternative, although a bit too fancy imo, is to use a 2d convolution with this kernel
# kernel = np.array([[1, 1, 1],
#                    [1, 0, 1],
#                    [1, 1, 1]])
# and sliding across this grid `mine_grid = df['has_mine'].values.reshape(y_size, x_size)`

# Less fancy although still better alternative would be to istead of iterating through all
# the positions instead iterate through all shifts (-1, 0), (+1, 0), ... of neighbors we
# consider and then roll through with that, basically we compute the convolution in this case
# by hand (note that every 1 in the convolution kernel corresponds to a shift
# but also don't want to implement this variant, I think my solution is, while slow, good enough for now.

adjacents = []
for x_curr in range(x_size):
    for y_curr in range(y_size):
        x_cond = (df.x - x_curr).abs() <= 1
        y_cond = (df.y - y_curr).abs() <= 1
        adjacents.append(df[x_cond & y_cond].has_mine.sum())

df["adjacent"] = adjacents

**54**. For rows of the DataFrame that contain a mine, set the value in the `'adjacent'` column to NaN.

In [5]:
df.loc[df.has_mine == 1, "adjacent"] = np.nan

**55**. Finally, convert the DataFrame to grid of the adjacent mine counts: columns are the `x` coordinate, rows are the `y` coordinate.

In [6]:
grid_df = df.pivot(index="y", columns="x", values="adjacent")
grid_df


x,0,1,2,3,4
y,,,,,
0,NaN,4.0,NaN,NaN,NaN
1,NaN,NaN,3.0,4.0,3.0
2,3.0,3.0,1.0,1.0,NaN
3,NaN,1.0,0.0,1.0,1.0
